In [160]:
from collections import defaultdict
import numpy as np
import math
#graph data is represented by edges connecting vertices and having a given weight
#vertices is a hashmap: vertices[vertex]=[edge_1,...,edge_n] of all the edges connected to a given vertex
#edges is a hashmap: edges[edge]=(start,end)
#weights is a hashmap: weights[edge]=weight
class graph(object):
    def __init__(self,vertices,edges,weights):
        self.vertices=vertices
        self.edges=edges
        self.weights=weights
        self.embed={}
        self.degrees={}
        self.maxdeg=0
        self.decktransf={}
    
    def compute_degrees(self):
        for vertex in self.vertices.keys():
            self.degrees[vertex]=len(self.vertices[vertex])
            if self.degrees[vertex]>self.maxdeg:
                self.maxdeg=self.degrees[vertex]
        
    
    def find_sep(self,epsilon,var=0.00001): #check that we don't need eta
        self.compute_degrees()
        d=self.maxdeg
        beta=math.pi/(d+var)
        nu=-2*math.log(math.tan(beta/2))
        tau=1
        lowb=nu*(1+epsilon)/epsilon
        for edge in self.edges.keys():
            if lowb/self.weights[edge]>1:
                tau=max(tau,lowb/self.weights[edge])
        return beta,tau
    
    def cayleytransf(self,z):
        x,y=z
        x0=-2*y/((1-x)**2+y**2)
        y0=(1-x**2-y**2)/((1-x)**2+y**2)
        return (x0,y0)
        
    
    def findtransf(self,edge1,edge2): #we translate everything on the upper half-plane
        start1,end1=edge1
        start2,end2=edge2
        xs1,ys1=self.cayleytransf(start1) #xs,ys are the coordinates of the starting point of the edge 
        xe1,ye1=self.cayleytransf(end1)   #xe,ye are the coordinates of the endpoint of the edge
        xs2,ys2=self.cayleytransf(start2)
        xe2,ye2=self.cayleytransf(end2)
        if xs1!=xe1: #we distinguish the case in which they lie on a semicircle from when they lie on a vertical axis
            a1=(ys1**2-ye1**2)/(2*(xs1-xe1))+(xs1+xe1)/2 #a semicircle has the form (x-a)**2+y**2=c
            c1=(xs1-a1)**2+ys1**2
            if xs1<xe1:
                rep1=a1-math.sqrt(c1) #for a given semicircle, the edge gives an orientation from start to end
                att1=a1+math.sqrt(c1) #the orientation goes rep->start->end->att. rep and att are the x-coordinate
            else:
                rep1=a1+math.sqrt(c1)
                att1=a1-math.sqrt(c1)
        else: # a line has the form x=a
            a1=xs1
            c1=0
            if ys1<ye1:
                rep1=a1
            else:
                att1=a1
        if xs2!=xe2:
            a2=(ys2**2-ye2**2)/(2*(xs2-xe2))+(xs2+xe2)/2
            c2=(xs2-a2)**2+ys2**2
            if xs2<xe2:
                rep2=a2-math.sqrt(c2)
                att2=a2+math.sqrt(c2)
            else:
                rep2=a2+math.sqrt(c2)
                att2=a2-math.sqrt(c2)
        else:
            a2=xs2
            c2=0
            if ys2<ye2:
                rep2=0
            else:
                att2=0
        if c1!=0: #semicircle case, we find the transformation sending the semicircle to the y axis with the right orientation and the starpoint to i
            base1=((xs1-att1)**2+ys1**2)/((rep1-att1)*ys1)
            mob1=np.array([[base1,-base1*rep1],[1,-att1]])
        elif ys1<ye1: #vertical line case, we find the transformation sending the line to the y axis and the start point to i
            mob1=np.array([[1/ys1,-rep1/ys1],[0,1]]) 
        else:
            mob1=np.array([[1/ys1,-att1/ys1],[0,1]])
        if c2!=0:
            base2=((xs2-att2)**2+ys2**2)/((rep2-att2)*ys2)
            mob2=np.array([[base2,-base2*rep2],[1,-att2]])
        elif ys2<ye2:
            mob2=np.array([[1/ys2,-rep2/ys2],[0,1]])
        else:
            mob2=np.array([[1/ys2,-att2/ys2],[0,1]])
        mob2_1=np.linalg.inv(mob2) #find the inverse of the second
        transfplane=mob2_1.dot(mob1) #multiply the first for the inverse of the second
        cayley=np.array([[1,-1j],[1,1j]]) #send back to the disk model
        return cayley.dot(transfplane)
    
    def transform(self,mob,point):
        z=point[0]+1j*point[1]
        num=mob[0,0]*z+mob[0,1]
        den=mob[1,0]*z+mob[1,1]
        return (np.real(num/den),np.imag(num/den))
    
    def hypembed(self,start,epsilon): #need to adapt for edges connecting the same point
        beta,tau=self.find_sep(epsilon)
        queue=[(start,None)]
        pairings={}
        while queue:
            point,parent=queue[0]
            if parent==None:
                self.embed[point]=(0,0)
                angle=0
                degree=self.degrees[point]
                alpha=2*math.pi/degree-2*beta
                for edge in self.vertices[point]:
                    r=self.weights[edge]*tau
                    hypr=math.tanh(r/2)
                    for vertex in self.edges[edge]:
                        if vertex!=point:
                            if vertex not in self.embed.keys():
                                self.embed[vertex]=(hypr*math.cos(angle),hypr*math.sin(angle)) 
                                queue.append((vertex,edge))
                            elif edge not in pairings.keys():
                                pairings[edge]=((0,0),(hypr*math.cos(angle),hypr*math.sin(angle)))
                            elif edge not in self.decktransf.keys():
                                self.decktransf[edge]=self.findtransf(pairings[edge],((0,0),(hypr*math.cos(angle),hypr*math.sin(angle))))
                    angle+=alpha+beta
            else:
                point1,point2=self.edges[parent]
                degree=self.degrees[point]
                alpha=2*math.pi/degree-2*beta
                if point1!=point:
                    parentpoint=point1
                else:
                    parentpoint=point2
                r0=self.weights[parent]*tau
                hypr0=math.tanh(r0/2)
                mob=self.findtransf((self.embed[parentpoint],self.embed[point]),((hypr0,0),(0,0)))
                mob_inv=self.findtransf(((hypr0,0),(0,0)),(self.embed[parentpoint],self.embed[point]))
                angle=alpha+beta
                for edge in self.vertices[point]:
                    if edge!=parent:
                        r=self.weights[edge]*tau
                        hypr=math.tanh(r/2)
                        for vertex in self.edges[edge]:
                            if vertex!=point:
                                if vertex not in self.embed.keys():
                                    self.embed[vertex]=self.transform(mob_inv,(hypr*math.cos(angle),hypr*math.sin(angle)))
                                    queue.append((vertex,edge))
                                elif edge not in pairings.keys():
                                    newcoord=self.transform(mob_inv,(hypr*math.cos(angle),hypr*math.sin(angle)))
                                    pairings[edge]=(self.embed[point],newcoord)
                                elif edge not in self.decktransf.keys():
                                    newcoord=self.transform(mob_inv,(hypr*math.cos(angle),hypr*math.sin(angle)))
                                    self.decktransf[edge]=self.findtransf(pairings[edge],(newcoord,self.embed[point]))
                        angle+=beta+alpha
            del queue[0]
            
                        
                
        
            
        

In [161]:
vertices={'A':['AB','AC'],'B':['AB','BC'],'C':['AC','BC']}
edges={'AB':('A','B'),'AC':('A','C'),'BC':('B','C')}
weights={'AB':1,'AC':1,'BC':1}

In [162]:
trigraph=graph(vertices,edges,weights)

In [163]:
trigraph.compute_degrees()
print(trigraph.degrees)

{'A': 2, 'B': 2, 'C': 2}


In [164]:
trigraph.hypembed('A',0.00001)

In [166]:
print(trigraph.embed,trigraph.decktransf)

{'A': (0, 0), 'B': (0.6557964407537916, 0.0), 'C': (-5.150587448263726e-06, 0.6557964407335654)} {'BC': array([[ 0.50000338-0.5000005j, -0.32789768-0.3279013j],
       [ 0.50000338+0.5000005j, -0.32789768+0.3279013j]])}
